Априори - это восходящий подход. Мы начали с каждого отдельного элемента в списке наборов элементов. Затем кандидаты генерируются путем объединения друг с другом. Мы увеличиваем длину наборов элементов по одному элементу за раз. Проверка подмножества выполняется на каждом этапе, и наборы элементов, содержащие нечастые подмножества, удаляются.
Мы повторяем процесс до тех пор, пока из данных не перестанут быть получены успешные наборы элементов.

In [2]:
import pandas as pd
data = pd.read_excel("./Online_Retail.xlsx")

In [3]:
data["Country"].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [3]:
#Убираем лишние пробелы
data['Description'] = data['Description'].str.strip()
data['Description'] = data['Description'].str.replace("'","")
#Убираем те транзакции которые были отмерены или те, номера которых мы не знаем
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]
data.head(3)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


In [4]:
#One_hot_encoding
def one_or_zero(x):
    if x < 1:
        return 0 
    return 1

country_dict = {}
for country in data["Country"].unique():
    country_dict[country] = (data[data["Country"]==country]
                            .groupby(['InvoiceNo', 'Description'])['Quantity']
                            .sum().unstack().reset_index().fillna(0)
                            .set_index('InvoiceNo')).applymap(one_or_zero)


In [5]:
country_dict["France"].columns

Index([' 50'S CHRISTMAS GIFT BAG LARGE', ' DOLLY GIRL BEAKER',
       ' I LOVE LONDON MINI BACKPACK', ' NINE DRAWER OFFICE TIDY',
       ' SET 2 TEA TOWELS I LOVE LONDON ', ' SPACEBOY BABY GIFT SET',
       ' TRELLIS COAT RACK', '10 COLOUR SPACEBOY PEN',
       '12 COLOURED PARTY BALLOONS', '12 EGG HOUSE PAINTED WOOD',
       ...
       'WRAP VINTAGE PETALS  DESIGN', 'YELLOW COAT RACK PARIS FASHION',
       'YELLOW GIANT GARDEN THERMOMETER', 'YELLOW SHARK HELICOPTER',
       'ZINC  STAR T-LIGHT HOLDER ', 'ZINC FOLKART SLEIGH BELLS',
       'ZINC HERB GARDEN CONTAINER', 'ZINC METAL HEART DECORATION',
       'ZINC T-LIGHT HOLDER STAR LARGE', 'ZINC T-LIGHT HOLDER STARS SMALL'],
      dtype='object', name='Description', length=1565)

In [6]:
a = "`POPPY'S PLAYHOUSE KITCHEN`__`POPPY'S PLAYHOUSE BEDROOM`"
print(a)


`POPPY'S PLAYHOUSE KITCHEN`__`POPPY'S PLAYHOUSE BEDROOM`


In [69]:
country_dict["France"].query("`10 COLOUR SPACEBOY PEN` == 1 and `12 COLOURED PARTY BALLOONS` == 1")

def gen_zap(items:str):
    items =  items.split("__")
    print(f""" {" and ".join([f"`{i}` == 1" for i in  items])}""")
    return f""" {" and ".join([f"`{i}` == 1" for i in  items])}"""

In [7]:
import pandas as pd
data = {"col1":[1,0,0,1], "col2":[1,1,1,1], "col3":[1,1,1,0], "col4":[0,1,0,1], "col5":[0,1,1,1]}
data = pd.DataFrame.from_dict(data, orient='index', columns=['A', 'B', 'C', 'D'])
data.head(20)

,A,B,C,D
col1,1,0,0,1
col2,1,1,1,1
col3,1,1,1,0
col4,0,1,0,1
col5,0,1,1,1


# Apriori

In [9]:
#Работает долго как зараза. 
#Доля транзакций содержащих набор элементов
def support(number_tranz_with_item, number_tranz):
    return number_tranz_with_item / number_tranz

#Как часто в транзакциях с множеством X встречается доп элемент y
def confidence(tranz_with_X_y, tranz_with_X):
    return tranz_with_X_y / tranz_with_X


def from_str_to_set(item):
    return set(item.split("__"))

def gen_zap(items:str):
    return " and ".join([f"`{i}` == 1" for i in items.split("__")])

def del_cur_item(items:str, cur_item:str):
    return items.replace(cur_item + "__", "").replace("__" + cur_item, "")


def not_in_blackset(pretender, blackset):
    for black_item in blackset:
        black_item = from_str_to_set(black_item)
        if black_item == black_item & pretender:
            return False
    return True 

def gen_cur_checkset(blackset:set, curent_itemset:set, basic_items:set, basic_items_sup:dict):
    new_checkset = []
    for current_items in curent_itemset:
        current_items = from_str_to_set(current_items)

        for item in basic_items:
            if item not in current_items:
                pretender = current_items | set([item])
                if not_in_blackset(pretender, blackset):
                    
                    new_checkset.append(["__".join(sorted(pretender)), basic_items_sup[item], item])

    return new_checkset


def first_scan(data:pd.DataFrame, min_supp:float):
    numb_tranz = data.shape[0]
    basic_items = set()
    blackset = set()
    supp_list = {}

    for column in data.columns:
        col_sup = support(data[column].sum(), numb_tranz)
        if col_sup >= min_supp:
            basic_items.update([column])
            supp_list[column] = col_sup
        else:
            blackset.update([column])
    return basic_items, blackset, supp_list



def apriori(data:pd.DataFrame, min_supp:float):
    basic_items, blackset, basic_sup_list = first_scan(data, min_supp)
    numb_tranz = data.shape[0]
    supp_list = {}
    u = 0
    current_checkset = gen_cur_checkset(blackset=blackset, curent_itemset=basic_items, basic_items=basic_items, basic_items_sup=basic_sup_list)
    while current_checkset:
        print(u)
        next_checkset = set()
        
        for items, base_item_sup, cur_item in current_checkset:
            #Проверяем support для добавления в элементы, которые будем проверять дальше
            items_sup =  support(data.query(gen_zap(items=items)).shape[0], numb_tranz)
            if items_sup >= min_supp:
                next_checkset.update([items])
                supp_list[f'{del_cur_item(items, cur_item)} --> {cur_item}'] = confidence(items_sup, base_item_sup)
            else:
                blackset.update([items])
        
        u+=1

        current_checkset = gen_cur_checkset(blackset=blackset, curent_itemset=next_checkset, basic_items=basic_items, basic_items_sup=basic_sup_list)
    print(basic_sup_list)
    return supp_list

print(dict(sorted(apriori(country_dict["France"], min_supp=0.1).items(), key=lambda x: x[1], reverse=True)))

0
{'LUNCH BAG APPLE DESIGN': 0.10629067245119306, 'LUNCH BAG RED RETROSPOT': 0.1301518438177874, 'LUNCH BAG SPACEBOY DESIGN ': 0.1019522776572668, 'LUNCH BOX WITH CUTLERY RETROSPOT ': 0.12147505422993492, 'PLASTERS IN TIN CIRCUS PARADE ': 0.14316702819956617, 'PLASTERS IN TIN SPACEBOY': 0.11713665943600868, 'PLASTERS IN TIN WOODLAND ANIMALS': 0.14533622559652928, 'POSTAGE': 0.6507592190889371, 'RABBIT NIGHT LIGHT': 0.16052060737527116, 'RED RETROSPOT MINI CASES': 0.11713665943600868, 'RED TOADSTOOL LED NIGHT LIGHT': 0.1540130151843818, 'REGENCY CAKESTAND 3 TIER': 0.10629067245119306, 'ROUND SNACK BOXES SET OF4 WOODLAND ': 0.13449023861171366, 'SET/20 RED RETROSPOT PAPER NAPKINS ': 0.11279826464208242, 'SET/6 RED SPOTTY PAPER CUPS': 0.11713665943600868, 'SET/6 RED SPOTTY PAPER PLATES': 0.10845986984815618, 'SPACEBOY LUNCH BOX ': 0.10629067245119306, 'STRAWBERRY LUNCH BOX WITH CUTLERY': 0.10412147505422993}
{'SET/6 RED SPOTTY PAPER CUPS --> SET/6 RED SPOTTY PAPER PLATES': 0.96, 'POSTAGE 

In [10]:
from itertools import compress
import pandas as pd
#Попробовать реализовать на дереве
class Node:
    def __init__(self, name:str, val:int = 0):
        self.val = val
        self.name = name
        #Все уникальные дети которые у него есть, возможно нужен не сет, а словарь
        # И словарь реально нужен
        self.childs = {}
        #Можно будет на каждой итерации проверять наличие ребёнка, которого мы сейчас ищем
        #И в последствии, если он есть, прибавлять его количество.
        self.downdict = {}

    #TODO ПРОВЕРИТЬ СТОИТ ЛИ ПЕРЕВОДИТЬ В СЕТЫ ИЛИ НЕТ!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #Спойлер - стоит, так как поиск в сете выполняется гораздо быстрее чем поиск в листе или словаре. 

    def update(self, childs:tuple):
        #Прибавляем количеству элементов в этой ветке 1
        self.val += 1
        flag_upd = False
        #Если остались ещё элементы в списке, то приписываем ребёнка и все оставшиеся элементы вниз по дереву
        if childs:
            flag_upd = True

            if self.childs.get(childs[0], 0) == 0:
                self.childs[childs[0]] = Node(name=childs[0])

            for child in childs:
                self.downdict[child] = self.downdict.get(child, 0) + 1
        
        return flag_upd
    

    def ppr(self, level:int):
        print(f"Level: {level} =====================================")
        print(f"Name: {self.name}\nVal: {self.val}")
        print(f"Childs: {[child.name for child in self.childs.values()]}")
        print(f"{self.downdict}\n")



# Дочерняя сущность, хранящая в себе свои связи, родителя, и значение.
# Можно добавить ещё все дочерние значения.
class FP_tree:
    def __init__(self, items:pd.DataFrame, min_supp: float = 0.1):
        self.root = Node("ROOT_TREE", 0)
        self.min_supp = min_supp * items.shape[0]
        self.items = items

        self.oko = items.sum().sort_values(ascending=False).to_dict()
        
                                                #Нужен ли тут sorted?
        self.oko = {item:val for item, val in sorted(self.oko.items(), key = lambda x: x[1]) if val >= min_supp }
        
        #Для последующей генерации правил задаём все существенные для нас имена
        self.rule_prime_dict = {name:{} for name in self.oko.keys()}

        #self.final_rule_dict = {name:set() for name in self.oko.keys()}
        self.final_rule_set = set()
        
    def _tree_update(self, childs:tuple, node:Node):
        flag_update = node.update(childs=childs)
        if flag_update:
            self._tree_update(childs=childs[1:], node = node.childs[childs[0]])

        return 0

    def grow(self):
        for i, row in self.items.iterrows():
            supp_items = sorted((item 
                                for item, val in row.to_dict().items() 
                                if (val and self.oko.get(item, 0))
                                ), 
                                key=lambda x: self.oko.get(x, 0), 
                                reverse=True)
            self._tree_update(childs=supp_items, node=self.root)

    def dict_sum(self, prime_dict:dict, add_dict:dict):
        for key, val in add_dict.items():
            prime_dict[key] = prime_dict.get(key, 0) + val

    def add_rule_to_item_node(self, node_name:str, down_dict:dict):
        #Потом для генерации финальных правил можно взять значение node_name (их количество) из oko
        self.dict_sum(self.rule_prime_dict[node_name], add_dict=down_dict)
    
    def gen_prime_rules(self, node:Node):
        #написать поиск в глубину для генерации правил
        if node.name!= "ROOT_TREE":
            self.add_rule_to_item_node(node_name=node.name, down_dict=node.downdict)
        for child in node.childs.values():
            self.gen_prime_rules(node = child)
        
        return 0
        
    def drop_min_supp(self):
        for item_rule in self.rule_prime_dict.keys():
                self.rule_prime_dict[item_rule] = (x for x in self.rule_prime_dict[item_rule].items() if x[1] > self.min_supp)

    def gen_final_rules(self):
        #Генерация primedict:
        self.gen_prime_rules(node=self.root)
        self.drop_min_supp()
        print(self.gen_prime_rules)
        #Генерация final_rule_dict
        for key in self.rule_prime_dict.keys():
            #print(key)
            key_val = self.oko.get(key, 0)
            if key_val:
                #если такой ключ есть в око, а следовательно под него существует правило
                names, values = list(zip(*self.rule_prime_dict[key], (key, key_val)))
                len_names = len(names)

                if int("1"*len_names, 2) > 2:
                    print(int("1"*len_names, 2))

                for i in range(3, int("1"*len_names, 2)):
                    #вот тут проверяем что число не является степенью двойки
                    #Если ноль - то является 
                    if i & i-1:
                        #создаём бинарную маску
                        mask = bin(i)[2:]
                        mask = "0"*(len_names- len(mask)) + mask
                        new_rule = frozenset(compress(names, map(int, list(mask))))
                        new_vals = compress(values, map(int, list(mask)))

                        # Вот тут можно заменить на  min(new_vals) / max(new_vals), чтобы увидеть вероятность покупки именно этого набора при покупке самого частого элемента в наборе
                        self.final_rule_set.update(((min(new_vals), new_rule),))
                        #Это для записи в dict, но работает ультрамедленно.
                        # for item in new_rule:
                        #     if (supp := min(values)) >= self.min_supp:
                        #         self.final_rule_dict[item].update(((supp, new_rule),))



    def ppr_tree(self, node:Node, level:int=0):
        node.ppr(level=level)
        for child in node.childs.values():
            self.ppr_tree(node=child, level = level+1)
        
        return 0
    

    

#TODO ДОБАВИТЬ КРАСИВЫЙ ВЫВОД ПРАВИЛ!
# Идти до того момента, пока сумма найденных элементов будет меньше downdict-а у корня!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
a = FP_tree(items = country_dict["France"], min_supp=0.1)
#a = FP_tree(items = data)
a.grow()
#a.ppr_tree(node=a.root)
print(len(a.oko))
a.gen_final_rules()

import sys
sys.getsizeof(a)
for val in a.final_rule_set:
    print(val)

1564
<bound method FP_tree.gen_prime_rules of <__main__.FP_tree object at 0x000002B911C95D00>>
3
127
(48, frozenset({'LUNCH BAG RED RETROSPOT', 'RABBIT NIGHT LIGHT', 'PLASTERS IN TIN WOODLAND ANIMALS'}))
(48, frozenset({'LUNCH BAG RED RETROSPOT', 'PLASTERS IN TIN WOODLAND ANIMALS', 'POSTAGE', 'RABBIT NIGHT LIGHT', 'ROUND SNACK BOXES SET OF4 WOODLAND '}))
(54, frozenset({'RED TOADSTOOL LED NIGHT LIGHT', 'PLASTERS IN TIN CIRCUS PARADE ', 'PLASTERS IN TIN WOODLAND ANIMALS', 'RABBIT NIGHT LIGHT'}))
(48, frozenset({'LUNCH BAG RED RETROSPOT', 'RED TOADSTOOL LED NIGHT LIGHT', 'PLASTERS IN TIN WOODLAND ANIMALS', 'RABBIT NIGHT LIGHT', 'PLASTERS IN TIN CIRCUS PARADE ', 'ROUND SNACK BOXES SET OF4 WOODLAND '}))
(48, frozenset({'LUNCH BAG RED RETROSPOT', 'PLASTERS IN TIN WOODLAND ANIMALS', 'ROUND SNACK BOXES SET OF4 WOODLAND '}))
(48, frozenset({'LUNCH BAG RED RETROSPOT', 'RED TOADSTOOL LED NIGHT LIGHT', 'PLASTERS IN TIN WOODLAND ANIMALS', 'PLASTERS IN TIN CIRCUS PARADE ', 'ROUND SNACK BOXES SET OF

# Далее идут служебные функции представляющие ценнсоть исключительно чтобы посмотреть как я пытался мыслить

In [20]:
from itertools import compress

d1 = {"c":4, "b":4, "a":3}
d2 = {"c":4, "b":1, "a":2, "d":4}
d3 = {"c":1, "g":1}

oko = {"c":9, "b":5, "a":5, "d":8, "f":5, "g":1}


prime_dict = {"f":{}, "d":{}}
node_name = "f"
node_val = 5
node_name_2 = "d"
node_val = 3
rules_dict = {"a":[], "b":[], "c":[], "d":[], "f":[], "g":[]}

def dict_sum(prime_dict:dict, add_dict):
    #fine
    for key, val in add_dict.items():
        prime_dict[key] = prime_dict.get(key, 0) + val

def add_rule_to_item_node(node_name:str, down_dict:dict, prime_dict:dict):
    #Потом можно взять значение node_name (их количество) из oko
    dict_sum(prime_dict[node_name], add_dict=down_dict)

add_rule_to_item_node(node_name="f", down_dict=d2, prime_dict=prime_dict)
add_rule_to_item_node(node_name="d", down_dict=d1, prime_dict=prime_dict)
add_rule_to_item_node(node_name="f", down_dict=d3, prime_dict=prime_dict)
print(prime_dict)


def gen_final_rules(prime_dict:dict, keys:list, oko:dict, rules_dict:dict):
    for key in keys:
        key_val = oko.get(key, 0)
        if key_val:
            #если такой ключ есть в око, а следовательно под него существует 
            names, values = list(zip(*prime_dict[key], (key, key_val)))
            len_names = len(names)
            for i in range(3, int("1"*len_names, 2)):
                #вот тут проверяем что число не является степенью двойки
                #Если ноль - то является 
                if i & i-1:
                    #создаём бинарную маску
                    mask = bin(i)[2:]
                    mask = "0"*(len_names- len(mask)) + mask
                    new_rule = set(compress(names, map(int, list(mask))))
                    for item in new_rule:
                        rules_dict[item].append((min(values), new_rule))

def drop_min_supp(prime_dict:dict, min_supp:float):
    for item_rule in prime_dict.keys():
            prime_dict[item_rule] = (x for x in prime_dict[item_rule].items() if x[1] > min_supp)


drop_min_supp(prime_dict=prime_dict, min_supp=2)
print(prime_dict)

gen_final_rules(prime_dict=prime_dict, keys=prime_dict.keys(), oko=oko, rules_dict=rules_dict)

print(rules_dict)
print(sys.getsizeof(rules_dict["a"]))


{'f': {'c': 5, 'b': 1, 'a': 2, 'd': 4, 'g': 1}, 'd': {'c': 4, 'b': 4, 'a': 3}}
{'f': <generator object drop_min_supp.<locals>.<genexpr> at 0x0000027124A0FBA0>, 'd': <generator object drop_min_supp.<locals>.<genexpr> at 0x0000027124A0FC10>}
{'a': [(3, {'a', 'd'}), (3, {'a', 'b'}), (3, {'a', 'b', 'd'}), (3, {'c', 'a'}), (3, {'c', 'a', 'd'}), (3, {'a', 'c', 'b'})], 'b': [(3, {'b', 'd'}), (3, {'a', 'b'}), (3, {'a', 'b', 'd'}), (3, {'c', 'b'}), (3, {'c', 'b', 'd'}), (3, {'a', 'c', 'b'})], 'c': [(4, {'f', 'c'}), (4, {'c', 'd'}), (3, {'c', 'd'}), (3, {'c', 'a'}), (3, {'c', 'a', 'd'}), (3, {'c', 'b'}), (3, {'c', 'b', 'd'}), (3, {'a', 'c', 'b'})], 'd': [(4, {'f', 'd'}), (4, {'c', 'd'}), (3, {'a', 'd'}), (3, {'b', 'd'}), (3, {'a', 'b', 'd'}), (3, {'c', 'd'}), (3, {'c', 'a', 'd'}), (3, {'c', 'b', 'd'})], 'f': [(4, {'f', 'd'}), (4, {'f', 'c'})], 'g': []}
120


In [23]:
from itertools import compress
a, b= list(zip(*prime_dict["f"].items(), ("haha", 1000)))
print(a)
print(b)
set(compress(a, map(int, list("011"))))

('c', 'b', 'a', 'd', 'g', 'haha')
(5, 1, 2, 4, 1, 1000)


{'a', 'b'}

In [13]:
p = country_dict["France"].sum().sort_values(ascending=False).to_dict()
# p = {i:val for i, val in p.items() if val>100}

# a = country_dict["France"].iloc[0]
# a = [item for item, val in a.to_dict().items() if (val and p.get(item, 0))] 
# a.sort(key=lambda x: p.get(x, 0), reverse=True)

        

def trrrr(a):
    a["haha"] = {"val":1}

f = {"lol":{"val":1, "lol2":2}}
trrrr(f["lol"])
print(f)

def grow_tree(tree:dict, sorted_item_list:list, deep:int, cur_step:int = 0):
        if deep > cur_step:
            if tree.get(sorted_item_list[cur_step], 0):
                tree[sorted_item_list[cur_step]]["val"] += 1
            else:
                tree[sorted_item_list[cur_step]] = {"val":1}
            
            grow_tree(tree=tree[sorted_item_list[cur_step]], sorted_item_list=sorted_item_list, deep=deep, cur_step = cur_step+1 )
            
        return 0
sorted_item_list=["lol", "haha", "hahahahahaha", "hahahahahaha", "test", "lol"]
grow_tree(tree=f, sorted_item_list=sorted_item_list,  deep=len(sorted_item_list))
print(f)
sorted_item_list=["lol", "haha", "hahahahahaha", "test", "lol"]
grow_tree(tree=f, sorted_item_list=sorted_item_list,  deep=len(sorted_item_list))
print(f)

NameError: name 'country_dict' is not defined

In [26]:
#Посмотреть как можно превратить в словарь так, чтобы оставить только ненулевые элементы.
def grow_tree(tree:dict, sorted_item_list:list, deep:int, cur_step:int = 0):
        if deep > cur_step:
            if tree.get(sorted_item_list[cur_step], 0):
                tree[sorted_item_list[cur_step]]["val"] += 1
            else:
                tree[sorted_item_list[cur_step]] = {"val":1}
            
            grow_tree(tree=tree[sorted_item_list[cur_step]], sorted_item_list=sorted_item_list, deep=deep, cur_step = cur_step+1 )
        return 0

def search_item_routs_in_tree(tree:dict, item:str, oko:dict):
    now_items = list(tree.keys()).sort(key=lambda x: oko.get(x, 0), reverse=True)
    now_items = [key for key in now_items[:now_items.find(item)]]


def FP_Growth(items:pd.DataFrame, min_supp:float):
    oko = items.sum().sort_values(ascending=False).to_dict()
    lenght = items.shape[0]
    oko = {item:val for item, val in sorted(oko.items(), key= lambda x: x[1]) if val >= min_supp * lenght}
    tree = {}

    for i, row in items.iterrows():
        supp_items = sorted((item 
                             for item, val in row.to_dict().items() 
                             if (val and oko.get(item, 0))
                             ), 
                             key=lambda x: oko.get(x, 0), 
                             reverse=True)

        grow_tree(tree=tree, sorted_item_list=supp_items,  deep=len(supp_items))
    
         
    return tree

FP_Growth(items=country_dict["France"], min_supp=0.0002).keys()
a=1
#Осталось написать генерацию паттернов на основе минимальной поддержки, так же подъёма по дереву.

